In [1]:
import numpy as np
import json
from tqdm import tqdm

In [6]:
class2mergedclass = {}
classes = set()
with open('../data/VOCdevkit/VOC2010/Classes/part_classes_merged.txt', 'r') as f:
    class_ind = 0
    line = f.readline().replace('\n','').strip()
    classes.add(line)
    merged_class = line + '::' + str(class_ind)
    while True:
        line = f.readline().replace('\n','').strip()
        if line == 'END_OF_FILE':
            break
        if line == '':
            class_ind += 1
            line = f.readline().replace('\n','').strip()
            classes.add(line)
            merged_class = line + '::' + str(class_ind)
            continue
        class2mergedclass[line] = merged_class

In [7]:
for k,v in class2mergedclass.items():
    print(k, v)

__background__ __background__::0
engine_1 ENGINE::1
engine_2 ENGINE::2
engine_3 ENGINE::3
engine_4 ENGINE::4
engine_5 ENGINE::5
engine_6 ENGINE::6
body BODY::7
beak FACE::8
hair FACE::8
head FACE::8
nose FACE::8
lear FACE::8
lebrow FACE::8
leye FACE::8
mouth FACE::8
rear FACE::8
rebrow FACE::8
reye FACE::8
lbleg LEG::9
lblleg LEG::9
lbuleg LEG::9
lbho LEG::9
lfleg LEG::10
lflleg LEG::10
lfuleg LEG::10
lfho LEG::10
lleg LEG::11
llleg LEG::11
luleg LEG::11
lfoot LEG::11
rbleg LEG::12
rblleg LEG::12
rbuleg LEG::12
rbho LEG::12
rfleg LEG::13
rflleg LEG::13
rfuleg LEG::13
rfho LEG::13
rleg LEG::14
rlleg LEG::14
ruleg LEG::14
rfoot LEG::14
neck NECK::15
lhand ARMHAND::16
llarm ARMHAND::16
luarm ARMHAND::16
rhand ARMHAND::17
rlarm ARMHAND::17
ruarm ARMHAND::17
plant PLANT::18
pot POT::19
tail TAIL::20
torso TORSO::21
cbackside_1 TRAIN_COACH::22
cbackside_2 TRAIN_COACH::22
cfrontside_1 TRAIN_COACH::23
cfrontside_2 TRAIN_COACH::23
cfrontside_3 TRAIN_COACH::23
cfrontside_4 TRAIN_COACH::23
cfront

In [10]:
classes = sorted(list(classes))
classes = [classes[-1]] + classes[:-1]
print(classes)

['__background__', 'ARMHAND', 'BODY', 'DOOR', 'ENGINE', 'FACE', 'LEG', 'NECK', 'PLANT', 'POT', 'SCREEN', 'TAIL', 'TORSO', 'TRAIN_COACH', 'TRAIN_HEAD', 'VEHICLE_SIDE', 'WHEEL', 'WINDOWS', 'WINGS']


In [39]:
filenames = np.loadtxt('../data/VOCdevkit/VOC2010/ImageSets/Main/trainval.txt', dtype=str)

In [39]:
for filename in tqdm(filenames):
    img_path = '../data/VOCdevkit/VOC2010/JPEGImages/%s.jpg' % filename
    anno_path = '../data/VOCdevkit/VOC2010/Annotations_Part_json/%s.json' % filename
    target = json.load(open(anno_path, 'r'))
    
    for obj in target['object']:
        new_part_anno = {v: {'xmin':[], 'ymin':[], 'xmax':[], 'ymax':[]} for v in class2mergedclass.values()}
        for part in obj['parts']:
            classname = part['name']
            if classname not in class2mergedclass:
                continue
            xmin = part['bndbox']['xmin']
            ymin = part['bndbox']['ymin']
            xmax = part['bndbox']['xmax']
            ymax = part['bndbox']['ymax']
            mergedclassname = class2mergedclass[classname]
            new_part_anno[mergedclassname]['xmin'].append(xmin)
            new_part_anno[mergedclassname]['ymin'].append(ymin)
            new_part_anno[mergedclassname]['xmax'].append(xmax)
            new_part_anno[mergedclassname]['ymax'].append(ymax)    
        obj['parts'] = [ {'name': k.split('::')[0], 'bndbox': {'xmin':min(v['xmin']), 'ymin':min(v['ymin']), 'xmax':max(v['xmax']), 'ymax':max(v['ymax'])}} for k,v in new_part_anno.items() if len(v['xmin']) > 0 ]
    
    json.dump(target, open('../data/VOCdevkit/VOC2010/Annotations_Part_json_merged_part_classes/%s.json' % filename, 'w'))

100%|███████████████████████████████████████████████████████████████████████████| 10103/10103 [00:22<00:00, 456.40it/s]


In [13]:
class2ind = np.concatenate((np.array(classes).reshape(-1,1), np.arange(len(classes)).reshape(-1,1)), axis=1)

In [14]:
class2ind

array([['__background__', '0'],
       ['ARMHAND', '1'],
       ['BODY', '2'],
       ['DOOR', '3'],
       ['ENGINE', '4'],
       ['FACE', '5'],
       ['LEG', '6'],
       ['NECK', '7'],
       ['PLANT', '8'],
       ['POT', '9'],
       ['SCREEN', '10'],
       ['TAIL', '11'],
       ['TORSO', '12'],
       ['TRAIN_COACH', '13'],
       ['TRAIN_HEAD', '14'],
       ['VEHICLE_SIDE', '15'],
       ['WHEEL', '16'],
       ['WINDOWS', '17'],
       ['WINGS', '18']], dtype='<U14')

In [15]:
np.savetxt('part_mergedclass2ind.txt', class2ind, fmt='%s %s')